In [263]:
import pandas as pd
from pymongo import MongoClient
import pprint
import re
import numpy as np
import folium
from folium import IFrame
import os
import requests
from Functions.functions import findNear
import webbrowser
import branca.colormap
from dotenv import load_dotenv
load_dotenv()

if not "KEY" in os.environ:
    raise ValueError("You should pass a KEY")
KEY = os.environ["KEY"]

In [233]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

### Empresas con al menos 1.000.000$ de ingresos

In [234]:
companies = db.data_companies_clean.find({ 
    "total_money_raised":{
        "$gte": 1000000
    }
}
)

In [235]:
data_companies = pd.DataFrame(companies)

### Lista con las geoDescription

In [236]:
list_geo = []
for e in range(len(data_companies)):
    list_geo.append(data_companies.geoDescription[e]) 

### Aplico la función para saber el número de empresas que hay alrededor de cada una de ellas.

In [237]:
def findNear(list_geo, radio_meters):
        geopoint = list_geo
        return list(db.data_companies_clean.find({
        "geoDescription": {
         "$near": {
             "$geometry": geopoint,
             "$maxDistance": radio_meters
         }
       }
    }
    )
    ) 

radio_meters = 5000
list_number_offices = []
list_offices=[]
for i in range(len(data_companies)):
    num_offices =  findNear(list_geo[i], radio_meters)
    list_offices.append(num_offices)
    list_number_offices.append(len(num_offices))
data_companies['number of offices near'] = list_number_offices

In [238]:
data_companies.head()

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,geoDescription,latitude,longitude,city,country,number of offices near
0,5dd655028accc5aeaf32f6c0,AdaptiveBlue,tech,15.0,2007,24000000,"{'type': 'Point', 'coordinates': [-74.3372, 40...",40.801358,-74.337200,NYC,USA,1
1,5dd655028accc5aeaf32f6c1,Livestream,tech,120.0,2007,14700000,"{'type': 'Point', 'coordinates': [-73.995625, ...",40.726155,-73.995625,New York,USA,42
2,5dd655028accc5aeaf32f6c2,Seesmic,tech,13.0,2007,16000000,"{'type': 'Point', 'coordinates': [-122.419204,...",37.775196,-122.419204,San Francisco,USA,80
3,5dd655028accc5aeaf32f6c5,MeetMoi,social,15.0,2007,5580000,"{'type': 'Point', 'coordinates': [-73.985506, ...",40.757929,-73.985506,New York City,USA,45
4,5dd655028accc5aeaf32f6c7,Scrybe,enterprise,20.0,2007,1310000,"{'type': 'Point', 'coordinates': [-74.761403, ...",40.270666,-74.761403,San Francisco,USA,1


### Necesito valores para hacer una clasificación de empresas y poder elegir una localización:
1.- Hago la media
2.- Empleados por oficina
3.- Una oficina cerca
4.- Hacer un ranking


In [239]:
data_companies['money by offices'] = data_companies['total_money_raised']/data_companies['number of offices near']

### Calculo la suma de empleados por oficina

In [240]:
def sumEmployees(list_offices):
    list_employee = []
    res=0
    for company in list_offices:
        lst=[]
        for i in company:
            lst.append(i['number_of_employees'])
        res+=i['number_of_employees']
        list_employee.append(lst)
    result = [sum(b) for b in list_employee]
    return result

In [241]:
data_companies['employee by offices'] = sumEmployees(list_offices)/data_companies['number of offices near']

### Filtro para que al menos tenga una oficina cerca

In [242]:
data_companies = data_companies[data_companies['number of offices near'] > 1]

### Hago un ranking de las columnas 

In [243]:
 data_companies['Rank_employee'] = data_companies['employee by offices'].rank()
data_companies['Rank_money'] = data_companies['money by offices'].rank()
data_companies['Rank_offices'] = data_companies['number of offices near'].rank()

data_companies.head()

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,geoDescription,latitude,longitude,city,country,number of offices near,money by offices,employee by offices,Rank_employee,Rank_money,Rank_offices
1,5dd655028accc5aeaf32f6c1,Livestream,tech,120.0,2007,14700000,"{'type': 'Point', 'coordinates': [-73.995625, ...",40.726155,-73.995625,New York,USA,42,3.500000e+05,50.000000,206.0,118.5,237.0
2,5dd655028accc5aeaf32f6c2,Seesmic,tech,13.0,2007,16000000,"{'type': 'Point', 'coordinates': [-122.419204,...",37.775196,-122.419204,San Francisco,USA,80,2.000000e+05,52.575000,251.0,80.0,292.0
3,5dd655028accc5aeaf32f6c5,MeetMoi,social,15.0,2007,5580000,"{'type': 'Point', 'coordinates': [-73.985506, ...",40.757929,-73.985506,New York City,USA,45,1.240000e+05,46.711111,187.0,51.0,251.5
5,5dd655028accc5aeaf32f6c8,Wakoopa,tech,8.0,2007,1000000,"{'type': 'Point', 'coordinates': [4.8948623, 5...",52.374523,4.894862,Amsterdam,NLD,6,1.666667e+05,12.000000,19.5,67.5,128.0
6,5dd655028accc5aeaf32f6c9,Dropbox,tech,350.0,2007,257000000,"{'type': 'Point', 'coordinates': [-122.4084994...",37.790943,-122.408499,San Francisco,USA,80,3.212500e+06,52.575000,251.0,272.0,292.0


### Hago un ranking total y ordeno

In [244]:
data_companies['ranking_total'] =  data_companies['Rank_money'] + data_companies['Rank_employee'] + data_companies['Rank_offices']


In [245]:
data_companies = data_companies.sort_values(['ranking_total'], ascending=False)
data_companies.head()

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,geoDescription,latitude,longitude,city,country,number of offices near,money by offices,employee by offices,Rank_employee,Rank_money,Rank_offices,ranking_total
47,5dd655028accc5aeaf32f6fb,Zynga,tech,115.0,2007,860000000,"{'type': 'Point', 'coordinates': [-122.404234,...",37.765158,-122.404234,San Francisco,USA,81,1.061728e+07,52.148148,221.0,315.0,322.0,858.0
370,5dd655028accc5aeaf32f8dd,Sunrun,tech,180.0,2007,337000000,"{'type': 'Point', 'coordinates': [-122.4041075...",37.787131,-122.404107,San Francisco,USA,80,4.212500e+06,52.575000,251.0,285.0,292.0,828.0
6,5dd655028accc5aeaf32f6c9,Dropbox,tech,350.0,2007,257000000,"{'type': 'Point', 'coordinates': [-122.4084994...",37.790943,-122.408499,San Francisco,USA,80,3.212500e+06,52.575000,251.0,272.0,292.0,815.0
216,5dd655028accc5aeaf32f7f6,Groupon,tech,10000.0,2008,114999999999,"{'type': 'Point', 'coordinates': [-87.643622, ...",41.896516,-87.643622,Chicago,USA,9,1.277778e+10,1126.666667,322.5,328.0,151.0,801.5
15,5dd655028accc5aeaf32f6d3,Ooyala,tech,300.0,2007,122000000,"{'type': 'Point', 'coordinates': [-122.084171,...",37.386665,-122.084171,Mountain View,USA,18,6.777778e+06,54.055556,278.5,304.0,215.0,797.5


In [246]:
data_companies = data_companies.reset_index(drop=True)

In [247]:
data_companies.to_csv('./data_companies_select.csv')

### Conexión con API de Google Places

In [248]:
BASE_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch"

### Lista de escuelas según una de las posiciones más rankeadas, en este caso he elegido Chicago.

In [249]:
res = requests.get("{}/json?location=37.765158, -122.404234&radius=1500&type=school&key={}".format(BASE_URL, KEY)).json()

In [250]:
coordinates_schools_latitude = []
coordinates_schools_longitude = []
name_schools = []
for i in range(len(res['results'])):
    coordinates_schools_latitude.append(res['results'][i]['geometry']['location']["lat"])
    coordinates_schools_longitude.append(res['results'][i]['geometry']['location']["lng"])
    name_schools.append(res['results'][i]['name'])
print(coordinates_schools_latitude, coordinates_schools_longitude, name_schools)

[37.7595222, 37.7662651, 37.7635349, 37.7646348, 37.7664402, 37.7730042, 37.762215, 37.7762375, 37.776527, 37.7745, 37.7676116, 37.7672933, 37.75495340000001, 37.7616204, 37.7720999, 37.7710801, 37.76116409999999, 37.76025120000001, 37.7726402, 37.7605811] [-122.4140747, -122.4020492, -122.3991487, -122.3970541, -122.4189417, -122.4135607, -122.404424, -122.4063687, -122.406369, -122.3988162, -122.4109711, -122.3910103, -122.4151875, -122.403887, -122.4093122, -122.4097826, -122.4112622, -122.3961201, -122.4099154, -122.3958437] ["John O'Connell Technical High School", 'World Gym', 'Live Oak School', 'Emily Coles Dance SF', 'Marshall Elementary School', 'Presidio Knolls School', 'Safety Training Seminars', 'Bessie Carmichael Elementary School', 'Bessie Carmichael Pre K', 'Academy of Art University - School of Fine Art Sculpture', 'Community Alliance-Special Ed', 'Minor Jr Daniel L', 'César Chávez Elementary School', 'Downtown High School', 'The Salvation Army Harbor House', 'Dynamic Dr

### Creo un Dataframe con las escuelas

In [251]:
dicc = {'latitude_school':coordinates_schools_latitude,
       'longitude_school': coordinates_schools_longitude,
       'name_school': name_schools}

In [252]:
dataframe_schools = pd.DataFrame(dicc)

In [253]:
display(dataframe_schools)

,latitude_school,longitude_school,name_school
0,37.759522,-122.414075,John O'Connell Technical High School
1,37.766265,-122.402049,World Gym
2,37.763535,-122.399149,Live Oak School
3,37.764635,-122.397054,Emily Coles Dance SF
4,37.766440,-122.418942,Marshall Elementary School
5,37.773004,-122.413561,Presidio Knolls School
6,37.762215,-122.404424,Safety Training Seminars
7,37.776238,-122.406369,Bessie Carmichael Elementary School
8,37.776527,-122.406369,Bessie Carmichael Pre K
9,37.774500,-122.398816,Academy of Art University - School of Fine Art...


In [254]:
data_companies.head()

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,geoDescription,latitude,longitude,city,country,number of offices near,money by offices,employee by offices,Rank_employee,Rank_money,Rank_offices,ranking_total
0,5dd655028accc5aeaf32f6fb,Zynga,tech,115.0,2007,860000000,"{'type': 'Point', 'coordinates': [-122.404234,...",37.765158,-122.404234,San Francisco,USA,81,1.061728e+07,52.148148,221.0,315.0,322.0,858.0
1,5dd655028accc5aeaf32f8dd,Sunrun,tech,180.0,2007,337000000,"{'type': 'Point', 'coordinates': [-122.4041075...",37.787131,-122.404107,San Francisco,USA,80,4.212500e+06,52.575000,251.0,285.0,292.0,828.0
2,5dd655028accc5aeaf32f6c9,Dropbox,tech,350.0,2007,257000000,"{'type': 'Point', 'coordinates': [-122.4084994...",37.790943,-122.408499,San Francisco,USA,80,3.212500e+06,52.575000,251.0,272.0,292.0,815.0
3,5dd655028accc5aeaf32f7f6,Groupon,tech,10000.0,2008,114999999999,"{'type': 'Point', 'coordinates': [-87.643622, ...",41.896516,-87.643622,Chicago,USA,9,1.277778e+10,1126.666667,322.5,328.0,151.0,801.5
4,5dd655028accc5aeaf32f6d3,Ooyala,tech,300.0,2007,122000000,"{'type': 'Point', 'coordinates': [-122.084171,...",37.386665,-122.084171,Mountain View,USA,18,6.777778e+06,54.055556,278.5,304.0,215.0,797.5


### Conecto con la API de Google Places para que me devuelva localizaciones de Starbucks

In [255]:
res = requests.get("{}/json?location=37.765158, -122.404234&radius=1500&type=cafe&key={}&name=Starbucks".format(BASE_URL, KEY))

In [256]:
res = res.json()

### Paso los Starbucks a un dataframe

In [257]:
coordinates_starbucks_latitude = []
coordinates_starbucks_longitude = []
for i in range(len(res['results'])):
    coordinates_starbucks_latitude.append(res['results'][i]['geometry']['location']["lat"])
    coordinates_starbucks_longitude.append(res['results'][i]['geometry']['location']["lng"])
print(coordinates_starbucks_latitude, coordinates_starbucks_longitude)

[37.766884, 37.7658845, 37.767102, 37.771251, 37.7770595, 37.7629128, 37.77867, 37.777035, 37.770242, 37.776819, 37.7711952] [-122.409623, -122.4034908, -122.393893, -122.389289, -122.417347, -122.4104227, -122.396701, -122.393641, -122.403458, -122.394074, -122.3892362]


In [258]:
dicc_starbucks = {'latitude_starbucks':coordinates_starbucks_latitude,
       'longitude_starbucks': coordinates_starbucks_longitude}
dataframe_starbucks = pd.DataFrame(dicc_starbucks)
display(dataframe_starbucks)

,latitude_starbucks,longitude_starbucks
0,37.766884,-122.409623
1,37.765884,-122.403491
2,37.767102,-122.393893
3,37.771251,-122.389289
4,37.777060,-122.417347
5,37.762913,-122.410423
6,37.778670,-122.396701
7,37.777035,-122.393641
8,37.770242,-122.403458
9,37.776819,-122.394074


### Mapa con la empresa en Rosa, las escuelas en Rojo, los Starbucks en verde y el resto de empresas en Azul

In [284]:
map_chicago = folium.Map(location=[37.765158,-122.404234], width=750, height=500, zoom_start=15)
for index, row in data_companies.iterrows():
    folium.Marker([row['latitude'], row['longitude']],
                        radius=7,
                        popup="Name company: {}. Money: {}. Employees{}".format(row['name'], row['total_money_raised'], row['number_of_employees']),
                        icon=folium.Icon(color='red', icon='briefcase'),
                        fill_color="#F45649", 
                       ).add_to(map_chicago)
for index, row in dataframe_schools.iterrows():
    folium.Marker([row['latitude_school'], row['longitude_school']],
                        radius=4,
                        popup="School: {}".format(row['name_school']),
                        icon=folium.Icon(icon='book'),
                        fill_color="#F35C50",
                       ).add_to(map_chicago)
for index, row in dataframe_starbucks.iterrows():
    folium.Marker([row['latitude_starbucks'], row['longitude_starbucks']],
                        radius=4,
                        icon=folium.Icon(color='darkgreen',icon='cutlery'),
                        fill_color="#4422DC",
                  popup="Starbucks"
                       ).add_to(map_chicago)
folium.Marker([37.765158,-122.404234],
                        radius=4,
                        icon=folium.Icon(color='pink',icon='heart-empty'),
                        popup="FILIPINOS Inc.",
                        fill_color="#F35C50"
                       ).add_to(map_chicago)
folium.Circle([37.765158, -122.404234],
                    fill_color='#de2314', color = 'grey', 
                    radius=300
                   ).add_to(map_chicago)



map_chicago.save('map_chicago.html')
webbrowser.open('map_chicago.html', new=2)
map_chicago